In [1]:
import os
import math
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import efficientnet_v2

# =============================
# Config
# =============================
DATA_ROOT = "/kaggle/input/stanford-dog/dataset/Images_split"  # đổi sang đường dẫn của bạn (train/val/test)
IMG_SIZE = 224
BATCH_SIZE = 16
NUM_CLASSES = 120
SEED = 42

# Lịch train
WARMUP_EPOCHS = 5
FINETUNE_EPOCHS = 30
WARMUP_LR = 1e-4
FT_LR = 2e-5  # hơi tăng nhẹ so với 2e-5
WEIGHT_DECAY = 2e-4
LABEL_SMOOTH = 0.10

# Mixed precision
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")
AUTOTUNE = tf.data.AUTOTUNE


2025-10-31 06:21:57.008603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761891717.230122      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761891717.296302      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data_augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomContrast(0.15),
    layers.RandomBrightness(0.10),
    layers.RandomZoom(0.10),
    # layers.RandomRotation(0.087),
    # layers.RandomTranslation(0.1, 0.1),
], name="data_augment")

preprocess = efficientnet_v2.preprocess_input

# =============================
# Dataset
# =============================
def _make_ds(split_dir, training=False):
    if not os.path.isdir(split_dir):
        raise FileNotFoundError(f"Missing directory: {split_dir}")
    ds = tf.keras.utils.image_dataset_from_directory(
        split_dir,
        labels='inferred',
        label_mode='int',
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        shuffle=training,
        seed=SEED,
    )
    if training:
        ds = ds.map(lambda x, y: (data_augment(x, training=True), y), num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x, y: (preprocess(x), y), num_parallel_calls=AUTOTUNE)
    if training:
        return ds.prefetch(AUTOTUNE)
    else:
        return ds.cache().prefetch(AUTOTUNE)

train_dir = os.path.join(DATA_ROOT, 'train')
val_dir = os.path.join(DATA_ROOT, 'val')
test_dir = os.path.join(DATA_ROOT, 'test')

train_ds = _make_ds(train_dir, training=True)
val_ds = _make_ds(val_dir, training=False)
try:
    test_ds = _make_ds(test_dir, training=False)
except Exception:
    test_ds = None


I0000 00:00:1761891732.512036      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 18473 files belonging to 120 classes.
Found 2641 files belonging to 120 classes.
Found 5280 files belonging to 120 classes.


In [3]:
def cbam_block_keras(x, reduction=8, name_prefix="cbam"):
    c = x.shape[-1]
    if c is None:
        raise ValueError("Channel dimension must be known for CBAM.")

    # Channel Attention
    avg_pool = layers.GlobalAveragePooling2D(keepdims=True, name=f"{name_prefix}_gap")(x)
    max_pool = layers.GlobalMaxPooling2D(keepdims=True, name=f"{name_prefix}_gmp")(x)

    hidden = max(1, int(c) // reduction)
    mlp = tf.keras.Sequential([
        layers.Conv2D(hidden, 1, activation='relu', padding='same'),
        layers.Conv2D(int(c), 1, activation='sigmoid', padding='same'),
    ], name=f"{name_prefix}_mlp")

    ca = layers.Add(name=f"{name_prefix}_chan_add")([mlp(avg_pool), mlp(max_pool)])
    x = layers.Multiply(name=f"{name_prefix}_chan_mul")([x, ca])

    # Spatial Attention
    avg_pool_sp = layers.Lambda(lambda t: tf.reduce_mean(t, axis=-1, keepdims=True), name=f"{name_prefix}_spat_avg")(x)
    max_pool_sp = layers.Lambda(lambda t: tf.reduce_max(t, axis=-1, keepdims=True), name=f"{name_prefix}_spat_max")(x)
    sa = layers.Concatenate(axis=-1, name=f"{name_prefix}_spat_cat")([avg_pool_sp, max_pool_sp])
    sa = layers.Conv2D(1, kernel_size=7, padding='same', activation='sigmoid', name=f"{name_prefix}_spat_conv")(sa)
    x = layers.Multiply(name=f"{name_prefix}_spat_mul")([x, sa])
    return x

In [4]:
def build_model(img_size=IMG_SIZE, num_classes=NUM_CLASSES, cbam_reduction=8, dropout_rate=0.4):
    inp = layers.Input(shape=(img_size, img_size, 3))
    base = efficientnet_v2.EfficientNetV2S(include_top=False, weights='imagenet', input_tensor=inp)

    x = base.output
    x = cbam_block_keras(x, reduction=cbam_reduction, name_prefix="cbam1")
    x = layers.GlobalAveragePooling2D(name="gap")(x)
    x = layers.BatchNormalization(name="bn_head")(x)
    x = layers.Dropout(dropout_rate, name="dropout_head")(x)
    out = layers.Dense(num_classes, activation='softmax', dtype='float32', name="probs")(x)

    model = models.Model(inp, out, name="EffNetV2S_CBAM_300")
    return model, base

model, base = build_model()


82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
def build_loss(num_classes=NUM_CLASSES, label_smooth=LABEL_SMOOTH):

    try:
        # Keras 3.3+ hỗ trợ label_smoothing trong Sparse loss
        return tf.keras.losses.SparseCategoricalCrossentropy(label_smoothing=label_smooth)
    except TypeError:
        # Fallback cho bản Keras cũ
        cce = tf.keras.losses.CategoricalCrossentropy(label_smoothing=label_smooth)
        def loss_fn(y_true, y_pred):
            y_true_oh = tf.one_hot(tf.cast(y_true, tf.int32), num_classes)
            return cce(y_true_oh, y_pred)
        return loss_fn


def compile_with(model, optimizer):

    loss = build_loss()
    metrics = [
        tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5")
    ]
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    print(f"✅ Model compiled with {optimizer.__class__.__name__}, "
          f"LR={optimizer.learning_rate if not hasattr(optimizer.learning_rate, '__call__') else 'schedule'}")
warmup_opt = tf.keras.optimizers.AdamW(learning_rate=WARMUP_LR, weight_decay=WEIGHT_DECAY)
compile_with(model, warmup_opt)

# ==== Fine-tune (CosineDecay) ====
steps_per_epoch = len(train_ds)
total_steps = steps_per_epoch * FINETUNE_EPOCHS

cosine_lr = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=FT_LR,
    decay_steps=total_steps,
    alpha=0.1  # 10% LR cuối cùng
)

ft_opt = tf.keras.optimizers.AdamW(learning_rate=cosine_lr, weight_decay=WEIGHT_DECAY)
compile_with(model, ft_opt)

✅ Model compiled with AdamW, LR=<Variable path=adamw/learning_rate, shape=(), dtype=float32, value=9.999999747378752e-05>
✅ Model compiled with AdamW, LR=1.9999999494757503e-05


In [6]:
# Freeze backbone (warmup)
for l in base.layers:
    l.trainable = False

compile_with(model, warmup_opt)  # ✅ phải truyền cả model và optimizer

# Callbacks (thống nhất monitor = val_accuracy)
ckpt_path = os.path.join("checkpoints", "effv2s_cbam_best.keras")
os.makedirs(os.path.dirname(ckpt_path), exist_ok=True)
log_dir = os.path.join("logs", "effv2s_cbam")
os.makedirs(log_dir, exist_ok=True)

cbs = [
    tf.keras.callbacks.ModelCheckpoint(
        ckpt_path, monitor='val_accuracy', mode='max',
        save_best_only=True, verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', mode='max',
        patience=7, restore_best_weights=True, verbose=1
    ),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=False),
]

print("\n[Warmup] Training head with backbone frozen.")
hist_warmup = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=WARMUP_EPOCHS,
    callbacks=cbs,
    verbose=1
)


✅ Model compiled with AdamW, LR=<Variable path=adamw/learning_rate, shape=(), dtype=float32, value=9.999999747378752e-05>

[Warmup] Training head with backbone frozen.
Epoch 1/5


I0000 00:00:1761891792.211509      58 service.cc:148] XLA service 0x789d400024c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761891792.212332      58 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1761891796.707494      58 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1761891821.841472      58 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1155/1155 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.3117 - loss: 3.6049 - top5: 0.4728
Epoch 1: val_accuracy improved from -inf to 0.90231, saving model to checkpoints/effv2s_cbam_best.keras
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 244s 150ms/step - accuracy: 0.3119 - loss: 3.6040 - top5: 0.4730 - val_accuracy: 0.9023 - val_loss: 1.2363 - val_top5: 0.9875
Epoch 2/5
1154/1155 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.8240 - loss: 1.4775 - top5: 0.9635
Epoch 2: val_accuracy improved from 0.90231 to 0.90761, saving model to checkpoints/effv2s_cbam_best.keras
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 112s 97ms/step - accuracy: 0.8240 - loss: 1.4775 - top5: 0.9635 - val_accuracy: 0.9076 - val_loss: 1.2040 - val_top5: 0.9905
Epoch 3/5
1154/1155 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.8451 - loss: 1.3936 - top5: 0.9721
Epoch 3: val_accuracy improved from 0.90761 to 0.91443, saving model to checkpoints/effv2s_cbam_best.keras
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 114s 98ms/step - accuracy: 0.8451 - lo

In [7]:
for l in base.layers:
    if any(tag in l.name for tag in ["block6", "block7", "top"]):
        l.trainable = not isinstance(l, tf.keras.layers.BatchNormalization)
    else:
        l.trainable = False

# Compile lại với optimizer fine-tune
compile_with(model, ft_opt)

print("\n[Finetune] Unfrozen block6–7; continue training...")
hist_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=FINETUNE_EPOCHS,
    callbacks=cbs,
    verbose=1
)


✅ Model compiled with AdamW, LR=1.9999999494757503e-05

[Finetune] Unfrozen block6–7; continue training...
Epoch 1/30
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.8800 - loss: 1.2360 - top5: 0.9874
Epoch 1: val_accuracy improved from 0.92048 to 0.92503, saving model to checkpoints/effv2s_cbam_best.keras
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 298s 160ms/step - accuracy: 0.8800 - loss: 1.2360 - top5: 0.9874 - val_accuracy: 0.9250 - val_loss: 1.1088 - val_top5: 0.9943
Epoch 2/30
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.9129 - loss: 1.1363 - top5: 0.9930
Epoch 2: val_accuracy improved from 0.92503 to 0.92995, saving model to checkpoints/effv2s_cbam_best.keras
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 119s 103ms/step - accuracy: 0.9129 - loss: 1.1363 - top5: 0.9930 - val_accuracy: 0.9300 - val_loss: 1.1007 - val_top5: 0.9955
Epoch 3/30
1155/1155 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.9279 - loss: 1.0911 - top5: 0.9955
Epoch 3: val_accuracy did not improve from 0.92995
1

In [8]:
# Eval & Save
# =============================
print("\n[Eval] Validation set:")
val_metrics = model.evaluate(val_ds, verbose=0, return_dict=True)
print({k: float(v) for k, v in val_metrics.items()})

if test_ds is not None:
    print("[Eval] Test set:")
    test_metrics = model.evaluate(test_ds, verbose=0, return_dict=True)
    print({k: float(v) for k, v in test_metrics.items()})
else:
    print("[Eval] Test set not provided. Skipping.")

final_path = os.path.join("artifacts", "effv2s_cbam_final.keras")
os.makedirs(os.path.dirname(final_path), exist_ok=True)
model.save(final_path)
print(f"Saved final model to: {final_path}")



[Eval] Validation set:
{'accuracy': 0.932601273059845, 'loss': 1.0730514526367188, 'top5': 0.9939416646957397}
[Eval] Test set:
{'accuracy': 0.9253787994384766, 'loss': 1.0848443508148193, 'top5': 0.9946969747543335}
Saved final model to: artifacts/effv2s_cbam_final.keras


In [9]:
print(f"[RESULT] Test accuracy: {test_metrics['accuracy']*100:.2f}%")


[RESULT] Test accuracy: 92.54%


In [10]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

def quick_eval(model, ds, name="Dataset"):
    loss, acc, top5 = model.evaluate(ds, return_dict=False, verbose=0)
    y_true, y_pred = [], []
    for x, y in ds:
        p = model.predict(x, verbose=0)
        y_true.append(y.numpy())
        y_pred.append(np.argmax(p, axis=1))
    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
    print(f"{name}: loss={loss:.4f}, acc={acc:.4f}, top5={top5:.4f}, F1={f1:.4f}")

print("\n📊 FINAL RESULTS")
quick_eval(model, train_ds, "Train")
quick_eval(model, val_ds, "Validation")
if 'test_ds' in locals() and test_ds is not None:
    quick_eval(model, test_ds, "Test")



📊 FINAL RESULTS
Train: loss=0.9271, acc=0.9870, top5=0.9999, F1=0.9876
Validation: loss=1.0731, acc=0.9326, top5=0.9939, F1=0.9316
Test: loss=1.0848, acc=0.9254, top5=0.9947, F1=0.9239
